**Преподаватель:** Юлия Пономарева

## **Задание**

**Цель:** изучить применение методов оптимизации для решения задачи классификации

**Описание задания:**

В домашнем задании необходимо применить полученные знания в теории оптимизации и машинном обучении для реализации логистической регрессии.

**Этапы работы:**

1. Загрузите данные. Используйте датасет с ирисами. Его можно загрузить непосредственно из библиотеки Sklearn. В данных оставьте только 2 класса: Iris Versicolor, Iris Virginica.
2. Самостоятельно реализуйте логистическую регрессию, без использования метода LogisticRegression из библиотеки. Можете использовать библиотеки pandas, numpy, math для реализации. Оформите в виде функции. Оформите в виде класса с методами.
3. Реализуйте метод градиентного спуска. Обучите логистическую регрессию этим методом. Выберете и посчитайте метрику качества. Метрика должна быть одинакова для всех пунктов домашнего задания. Для упрощения сравнения выберете только одну метрику.
4. Повторите п. 3 для метода скользящего среднего (Root Mean Square Propagation, RMSProp).
5. Повторите п. 3 для ускоренного по Нестерову метода адаптивной оценки моментов (Nesterov–accelerated Adaptive Moment Estimation, Nadam).
6. Сравните значение метрик для реализованных методов оптимизации. Можно оформить в виде таблицы вида |метод|метрика|время работы| (время работы опционально). Напишите вывод.

### 1. Загрузка данных и подготовка датасета:

In [542]:
# Импортируем необходимые библиотеки
from sklearn import datasets
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [543]:
# Загружаем датасет с ирисами
iris = datasets.load_iris()

In [544]:
# Выводим описание датасета
print(iris.DESCR)

.. _iris_dataset:

Iris plants dataset
--------------------

**Data Set Characteristics:**

    :Number of Instances: 150 (50 in each of three classes)
    :Number of Attributes: 4 numeric, predictive attributes and the class
    :Attribute Information:
        - sepal length in cm
        - sepal width in cm
        - petal length in cm
        - petal width in cm
        - class:
                - Iris-Setosa
                - Iris-Versicolour
                - Iris-Virginica
                
    :Summary Statistics:

    ============== ==== ==== ======= ===== ====================
                    Min  Max   Mean    SD   Class Correlation
    ============== ==== ==== ======= ===== ====================
    sepal length:   4.3  7.9   5.84   0.83    0.7826
    sepal width:    2.0  4.4   3.05   0.43   -0.4194
    petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
    petal width:    0.1  2.5   1.20   0.76    0.9565  (high!)
    ============== ==== ==== ======= ===== ===========

In [545]:
# Загрузка данных из датасета iris
print("Ключи словаря (iris.keys):", iris.keys())
print("\nДанные (iris.data):", iris.data[:5])
print("\nЦелевые значения (iris.target):", iris.target)
print("\nИмена классов (iris.target_names):", iris.target_names)
print("\nИмена признаков (iris.feature_names):", iris.feature_names)

Ключи словаря (iris.keys): dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])

Данные (iris.data): [[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]]

Целевые значения (iris.target): [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]

Имена классов (iris.target_names): ['setosa' 'versicolor' 'virginica']

Имена признаков (iris.feature_names): ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']


In [546]:
# Преобразуем датасет в DataFrame
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)


In [547]:
df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [548]:
# Эта строка присваивает столбцу 'target' в DataFrame `df` значения из массива `iris.target`
df['target'] = iris.target

In [549]:
df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [550]:
# принт уникальных значений target
unique_targets = df['target'].unique()
print(unique_targets)

[0 1 2]


In [551]:
# Ищем соответствие между метками классов и их именами
print("Метка класса 0:", iris.target_names[0])
print("Метка класса 1:", iris.target_names[1])
print("Метка класса 2:", iris.target_names[2])

Метка класса 0: setosa
Метка класса 1: versicolor
Метка класса 2: virginica


In [552]:
# Оставляем только два класса: Iris Versicolor (класс 1) и Iris Virginica (класс 2)
df = df[df['target'] != 0]

In [553]:
df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
50,7.0,3.2,4.7,1.4,1
51,6.4,3.2,4.5,1.5,1
52,6.9,3.1,4.9,1.5,1
53,5.5,2.3,4.0,1.3,1
54,6.5,2.8,4.6,1.5,1
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [554]:
# Признаки и целевая переменная
X = df.drop(['target'], axis=1)
y = df['target']

In [555]:
y = y - 1  # Преобразование меток классов (1 и 2) в (0 и 1)

In [556]:
y

50     0
51     0
52     0
53     0
54     0
      ..
145    1
146    1
147    1
148    1
149    1
Name: target, Length: 100, dtype: int64

In [557]:
# Разбиваем на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [558]:
# Масштабирование данных
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [559]:
# Преобразование y_train и y_test для обучения модели
y_train = np.where(y_train == 1, 1, 0)
y_test = np.where(y_test == 1, 1, 0)

In [560]:
# Добавим столбец "единиц" к обучающим данным для учета смещения
X_train = np.c_[np.ones((X_train.shape[0], 1)), X_train]
X_test = np.c_[np.ones((X_test.shape[0], 1)), X_test]

### 2. Реализация логистической регрессии в виде функции:

In [561]:
# Инициализация весов
def initialize_weights(n):
    """
    Инициализация весов модели логистической регрессии.

    Параметры:
    n (int): количество признаков

    Возвращает:
    w (array): вектор весов размерности n
    b (float): смещение
    """
    w = np.zeros(n)
    b = 0
    return w, b

# Сигмоидная функция
def sigmoid(z):
    """
    Сигмоидная функция для преобразования результата линейной комбинации в вероятность.

    Параметры:
    z (array): линейная комбинация признаков и весов

    Возвращает:
    output (array): вероятность принадлежности к классу 1
    """
    return 1 / (1 + np.exp(-z))

# Функция предсказания
def predict(X, w, b):
    """
    Функция для предсказания меток классов на основе входных данных и параметров модели.

    Параметры:
    X (array): матрица признаков
    w (array): вектор весов модели
    b (float): смещение

    Возвращает:
    y_pred (array): предсказанные вероятности принадлежности к классу 1
    """
    z = np.dot(X, w) + b
    y_pred = sigmoid(z)
    return y_pred

# Функция вычисления стоимости (функция потерь)
def compute_cost(y, y_pred):
    """
    Функция для вычисления стоимости (функции потерь) модели логистической регрессии.

    Параметры:
    y (array): истинные метки классов
    y_pred (array): предсказанные вероятности принадлежности к классу 1

    Возвращает:
    cost (float): значение функции потерь
    """
    m = len(y)
    epsilon = 1e-15
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
    cost = (-1/m) * np.sum(y * np.log(y_pred) + (1 - y) * np.log(1 - y_pred))
    return cost

# Функция градиентного спуска
def gradient_descent(X, y, learning_rate, iterations):
    """
    Функция для обучения модели логистической регрессии с помощью градиентного спуска.

    Параметры:
    X (array): матрица признаков
    y (array): истинные метки классов
    learning_rate (float): скорость обучения
    iterations (int): количество итераций

    Возвращает:
    w (array): обновленные веса модели
    b (float): обновленное смещение
    cost_history (list): история значений функции потерь на каждой итерации
    """
    m, n = X.shape
    w, b = initialize_weights(n)
    cost_history = []

    for i in range(iterations):
        y_pred = predict(X, w, b)

        dw = (1/m) * np.dot(X.T, (y_pred - y))
        db = (1/m) * np.sum(y_pred - y)

        w -= learning_rate * dw
        b -= learning_rate * db

        cost = compute_cost(y, y_pred)
        cost_history.append(cost)

        if i % 100 == 0:
            print(f"Iteration {i}: Cost {cost}")

    return w, b, cost_history

# Функция оценки точности модели
def evaluate_accuracy(X_test, y_test, w, b):
    """
    Функция для оценки точности модели логистической регрессии на тестовом наборе данных.

    Параметры:
    X_test (array): матрица признаков тестового набора
    y_test (array): истинные метки классов тестового набора
    w (array): вектор весов модели
    b (float): смещение модели

    Возвращает:
    accuracy (float): точность модели в процентах
    """
    y_pred = predict(X_test, w, b)
    y_pred_class = np.where(y_pred >= 0.5, 1, 0)  # преобразование вероятностей в метки классов
    accuracy = np.mean(y_pred_class == y_test) * 100  # вычисление точности в процентах
    return accuracy



Применим функцию градиентного спуска для обучения модели логистической регрессии на данном датасете с ирисами

In [562]:
# Обучение модели с помощью градиентного спуска
learning_rate = 0.1
iterations = 1000
w, b, cost_history = gradient_descent(X_train, y_train, learning_rate, iterations)

# Оценка точности модели на тестовом наборе данных
accuracy = evaluate_accuracy(X_test, y_test, w, b)
print(f"Accuracy: {accuracy}%")



Iteration 0: Cost 0.6931471805599454
Iteration 100: Cost 0.1814271304042798
Iteration 200: Cost 0.12364209699801042
Iteration 300: Cost 0.09774957810564053
Iteration 400: Cost 0.0827828855942761
Iteration 500: Cost 0.07290635966229642
Iteration 600: Cost 0.06583331542852558
Iteration 700: Cost 0.06047949132746119
Iteration 800: Cost 0.05626194610403769
Iteration 900: Cost 0.05283795451974021
Accuracy: 80.0%


1. Градиентный спуск:
   - Метод градиентного спуска показал результат в виде точности 80% на тестовом наборе данных после 1000 итераций обучения.
   - Значение функции стоимости уменьшается с каждой итерацией, указывая на сходимость модели.

### 3. Теперь оформим это в виде класса с методами.

In [563]:
class LogisticRegression:
    def __init__(self, learning_rate=0.01, iterations=1000):
        self.learning_rate = learning_rate
        self.iterations = iterations
        self.w = None
        self.b = None
        self.cost_history = []

    def initialize_weights(self, n):
        """
        Инициализация весов модели логистической регрессии.

        Параметры:
        n (int): количество признаков
        """
        self.w = np.zeros(n)
        self.b = 0

    @staticmethod
    def sigmoid(z):
        """
        Сигмоидная функция для преобразования результата линейной комбинации в вероятность.

        Параметры:
        z (array): линейная комбинация признаков и весов

        Возвращает:
        output (array): вероятность принадлежности к классу 1
        """
        return 1 / (1 + np.exp(-z))

    def predict(self, X):
        """
        Функция для предсказания меток классов на основе входных данных и параметров модели.

        Параметры:
        X (array): матрица признаков

        Возвращает:
        y_pred (array): предсказанные вероятности принадлежности к классу 1
        """
        z = np.dot(X, self.w) + self.b
        return self.sigmoid(z)

    @staticmethod
    def compute_cost(y, y_pred):
        """
        Функция для вычисления стоимости (функции потерь) модели логистической регрессии.

        Параметры:
        y (array): истинные метки классов
        y_pred (array): предсказанные вероятности принадлежности к классу 1

        Возвращает:
        cost (float): значение функции потерь
        """
        m = len(y)
        epsilon = 1e-15
        y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
        cost = (-1/m) * np.sum(y * np.log(y_pred) + (1 - y) * np.log(1 - y_pred))
        return cost

    def fit(self, X, y):
        """
        Функция для обучения модели логистической регрессии с помощью градиентного спуска.

        Параметры:
        X (array): матрица признаков
        y (array): истинные метки классов
        """
        m, n = X.shape
        self.initialize_weights(n)

        for i in range(self.iterations):
            y_pred = self.predict(X)

            dw = (1/m) * np.dot(X.T, (y_pred - y))
            db = (1/m) * np.sum(y_pred - y)

            self.w -= self.learning_rate * dw
            self.b -= self.learning_rate * db

            cost = self.compute_cost(y, y_pred)
            self.cost_history.append(cost)

            if i % 100 == 0:
                print(f"Iteration {i}: Cost {cost}")

    def evaluate_accuracy(self, X_test, y_test):
        """
        Функция для оценки точности модели логистической регрессии на тестовом наборе данных.

        Параметры:
        X_test (array): матрица признаков тестового набора
        y_test (array): истинные метки классов тестового набора

        Возвращает:
        accuracy (float): точность модели в процентах
        """
        y_pred = self.predict(X_test)
        y_pred_class = np.where(y_pred >= 0.5, 1, 0)  # преобразование вероятностей в метки классов
        accuracy = np.mean(y_pred_class == y_test) * 100  # вычисление точности в процентах
        return accuracy


In [564]:
# Создадим и обучим модель с использованием метода SGD
model_SGD = LogisticRegression(learning_rate=0.01, iterations=1000)
model_SGD.fit(X_train, y_train)

# Оценка точности модели на тестовом наборе данных
accuracy_SGD = model_SGD.evaluate_accuracy(X_test, y_test)
print(f"Accuracy: {accuracy_SGD}%")

Iteration 0: Cost 0.6931471805599454
Iteration 100: Cost 0.45908130152855275
Iteration 200: Cost 0.36677138337919923
Iteration 300: Cost 0.3150119513424012
Iteration 400: Cost 0.2802327747754729
Iteration 500: Cost 0.25446851085833455
Iteration 600: Cost 0.2342393427021266
Iteration 700: Cost 0.217745167515065
Iteration 800: Cost 0.20393893981888678
Iteration 900: Cost 0.19215774021617862
Accuracy: 90.0%


2. SGD:
   - Модель, обученная с использованием метода стохастического градиентного спуска (SGD), достигла наивысшей точности в 90% на тестовом наборе данных.
   - Функция стоимости также уменьшается с каждой итерацией, что свидетельствует о сходимости модели к оптимальным параметрам.

### 4. Метод RMSProp

In [565]:
class RMSProp:
    def __init__(self, learning_rate=0.01, decay_rate=0.9, epsilon=1e-7, iterations=1000):
        """
        Конструктор класса RMSProp.

        Параметры:
        - learning_rate (float): скорость обучения (по умолчанию 0.01)
        - decay_rate (float): коэффициент затухания (по умолчанию 0.9)
        - epsilon (float): эпсилон для численной стабильности (по умолчанию 1e-7)
        - iterations (int): количество итераций обучения (по умолчанию 1000)
        """
        self.learning_rate = learning_rate
        self.decay_rate = decay_rate
        self.epsilon = epsilon
        self.iterations = iterations
        self.w = None
        self.b = None
        self.cost_history = []
        self.cache_w = 0
        self.cache_b = 0

    def initialize_weights(self, n):
        """
        Инициализация весов модели.

        Параметры:
        - n (int): количество признаков во входном слое
        """
        self.w = np.zeros(n)
        self.b = 0

    def compute_gradient(self, X, y, y_pred):
        """
        Вычисление градиента стоимостной функции.

        Параметры:
        - X (array): матрица признаков
        - y (array): вектор истинных меток классов
        - y_pred (array): вектор предсказанных значений

        Возвращает:
        - dw (array): градиент по весам
        - db (float): градиент по смещению
        """
        m = len(y)
        dw = (1/m) * np.dot(X.T, (y_pred - y))
        db = (1/m) * np.sum(y_pred - y)
        return dw, db

    def predict(self, X):
        """
        Прогнозирование на основе входных данных.

        Параметры:
        - X (array): матрица признаков

        Возвращает:
        - y_pred (array): вектор предсказанных значений
        """
        return np.dot(X, self.w) + self.b

    def compute_cost(self, y, y_pred):
        """
        Вычисление стоимостной функции.

        Параметры:
        - y (array): вектор истинных меток классов
        - y_pred (array): вектор предсказанных значений

        Возвращает:
        - cost (float): значение стоимостной функции
        """
        return np.mean((y_pred - y)**2)

    def fit(self, X, y):
        """
        Обучение модели методом RMSProp.

        Параметры:
        - X (array): матрица признаков
        - y (array): вектор истинных меток классов
        """
        m, n = X.shape
        self.initialize_weights(n)

        for i in range(self.iterations):
            y_pred = self.predict(X)
            dw, db = self.compute_gradient(X, y, y_pred)

            self.cache_w = self.decay_rate * self.cache_w + (1 - self.decay_rate) * dw**2
            self.cache_b = self.decay_rate * self.cache_b + (1 - self.decay_rate) * db**2

            self.w -= (self.learning_rate / np.sqrt(self.cache_w + self.epsilon)) * dw
            self.b -= (self.learning_rate / np.sqrt(self.cache_b + self.epsilon)) * db

            cost = self.compute_cost(y, y_pred)
            self.cost_history.append(cost)

            if i % 100 == 0:
                print(f"Iteration {i}: Cost {cost}")

    def evaluate_accuracy(self, X_test, y_test):
        """
        Оценка точности модели на тестовом наборе данных.

        Параметры:
        - X_test (array): матрица признаков тестового набора
        - y_test (array): вектор истинных меток классов тестового набора

        Возвращает:
        - accuracy (float): точность модели в процентах
        """
        y_pred = self.predict(X_test)
        y_pred_class = np.where(y_pred >= 0.5, 1, 0)  # Преобразование вероятностей в метки классов
        accuracy = np.mean(y_pred_class == y_test) * 100
        return accuracy


In [566]:
# Создание и обучение модели
model_RMSProp = RMSProp(learning_rate=0.01, decay_rate=0.9, epsilon=1e-7, iterations=1000)
model_RMSProp.fit(X_train, y_train)

# Оценка точности на тестовом наборе
accuracy = model_RMSProp.evaluate_accuracy(X_test, y_test)
print(f"Test Accuracy: {accuracy}%")

Iteration 0: Cost 0.525
Iteration 100: Cost 0.04496702226353591
Iteration 200: Cost 0.04527152713307672
Iteration 300: Cost 0.0452734423830569
Iteration 400: Cost 0.04527344892722721
Iteration 500: Cost 0.045273448849999884
Iteration 600: Cost 0.04527344885016306
Iteration 700: Cost 0.04527344885016438
Iteration 800: Cost 0.04527344885016436
Iteration 900: Cost 0.04527344885016436
Test Accuracy: 85.0%


3. RMSProp:
   - При использовании метода RMSProp точность модели на тестовом наборе данных составила 85% после 1000 итераций обучения.
   - Функция стоимости сначала значительно уменьшается, достигая значения в районе 0.045, и затем стабилизируется, указывая на сходимость модели.

### 5. Метод Nadam

In [570]:
class Nadam:
    """
    Класс, реализующий оптимизацию методом Nadam.

    Параметры:
    - learning_rate: коэффициент скорости обучения
    - beta1: параметр для вычисления скользящего среднего градиента
    - beta2: параметр для вычисления скользящего среднего квадрата градиента
    - epsilon: значение для стабилизации деления
    - iterations: количество итераций обучения
    """

    def __init__(self, learning_rate=0.01, beta1=0.9, beta2=0.999, epsilon=1e-8, iterations=1000):
        """
        Инициализация параметров метода оптимизации Nadam.

        Параметры:
        - learning_rate: float, коэффициент скорости обучения
        - beta1: float, параметр для вычисления скользящего среднего градиента
        - beta2: float, параметр для вычисления скользящего среднего квадрата градиента
        - epsilon: float, значение для стабилизации деления
        - iterations: int, количество итераций обучения
        """
        self.learning_rate = learning_rate
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        self.iterations = iterations
        self.w = None
        self.b = None
        self.cost_history = []
        self.m_w = 0
        self.m_b = 0
        self.v_w = 0
        self.v_b = 0
        self.t = 0

    def initialize_weights(self, n):
        """
        Инициализация весов модели.

        Параметры:
        - n: int, количество признаков во входных данных X
        """
        self.w = np.zeros(n)
        self.b = 0

    def compute_gradient(self, X, y, y_pred):
        """
        Вычисление градиента модели.

        Параметры:
        - X: numpy array, входные данные
        - y: numpy array, целевая переменная
        - y_pred: numpy array, предсказанные значения

        Возвращает:
        Градиенты dw (по весам) и db (по смещению).
        """
        m = len(y)
        dw = (1/m) * np.dot(X.T, (y_pred - y))
        db = (1/m) * np.sum(y_pred - y)
        return dw, db

    def predict(self, X):
        """
        Предсказание результатов на основе входных данных X.

        Параметры:
        - X: numpy array, входные данные

        Возвращает:
        Предсказанные значения результатов.
        """
        return np.dot(X, self.w) + self.b

    def compute_cost(self, y, y_pred):
        """
        Вычисление функции стоимости.

        Параметры:
        - y: numpy array, целевая переменная
        - y_pred: numpy array, предсказанные значения

        Возвращает:
        Значение функции стоимости.
        """
        return np.mean((y_pred - y)**2)

    def fit(self, X, y):
        """
        Обучение модели методом Nadam.

        Параметры:
        - X: numpy array, входные данные
        - y: numpy array, целевая переменная
        """
        m, n = X.shape
        self.initialize_weights(n)

        for i in range(self.iterations):
            self.t += 1
            y_pred = self.predict(X)
            dw, db = self.compute_gradient(X, y, y_pred)

            self.m_w = self.beta1 * self.m_w + (1 - self.beta1) * dw
            self.m_b = self.beta1 * self.m_b + (1 - self.beta1) * db
            self.v_w = self.beta2 * self.v_w + (1 - self.beta2) * dw**2
            self.v_b = self.beta2 * self.v_b + (1 - self.beta2) * db**2

            m_w_hat = self.m_w / (1 - self.beta1**self.t)
            m_b_hat = self.m_b / (1 - self.beta1**self.t)
            v_w_hat = self.v_w / (1 - self.beta2**self.t)
            v_b_hat = self.v_b / (1 - self.beta2**self.t)

            self.w -= (self.learning_rate / (np.sqrt(v_w_hat) + self.epsilon)) * (self.beta1 * m_w_hat + (1 - self.beta1) * dw)
            self.b -= (self.learning_rate / (np.sqrt(v_b_hat) + self.epsilon)) * (self.beta1 * m_b_hat + (1 - self.beta1) * db)

            cost = self.compute_cost(y, y_pred)
            self.cost_history.append(cost)

            if i % 100 == 0:
                print(f"Iteration {i}: Cost {cost}")

    def evaluate_accuracy(self, X_test, y_test):
        """
        Оценка точности модели на тестовом наборе данных.

        Параметры:
        - X_test: numpy array, тестовые входные данные
        - y_test: numpy array, целевые значения тестовой выборки

        Возвращает:
        Точность модели на тестовом наборе данных.
        """
        y_pred = self.predict(X_test)
        y_pred_class = np.where(y_pred >= 0.5, 1, 0)
        accuracy = np.mean(y_pred_class == y_test) * 100
        return accuracy


In [571]:
# Создание и обучение модели с использованием метода Nadam
model_Nadam = Nadam(learning_rate=0.01, beta1=0.9, beta2=0.999, epsilon=1e-8, iterations=1000)
model_Nadam.fit(X_train, y_train)

# Оценка точности модели на тестовом наборе
accuracy_Nadam = model_Nadam.evaluate_accuracy(X_test, y_test)
print(f"Test Accuracy (Nadam): {accuracy_Nadam}%")


Iteration 0: Cost 0.525
Iteration 100: Cost 0.04514873399083481
Iteration 200: Cost 0.04490368989770834
Iteration 300: Cost 0.04488777626461323
Iteration 400: Cost 0.04488743240039901
Iteration 500: Cost 0.044887430000248044
Iteration 600: Cost 0.04488742999548042
Iteration 700: Cost 0.04488742999547848
Iteration 800: Cost 0.04488742999547847
Iteration 900: Cost 0.04488742999547847
Test Accuracy (Nadam): 80.0%


4. Nadam:
   - Метод оптимизации Nadam показал точность 80% на тестовом наборе данных после 1000 итераций обучения.
   - Значение функции стоимости также убывает с каждой итерацией, сходясь к оптимальным параметрам модели.

### 6. Теперь, обучим модели и сравним их по accuray:

In [569]:
import time

# Создадим и обучим модели с использованием всех трех методов

print("Метод оптимизации: {model_SGD}")
model_SGD = LogisticRegression(learning_rate=0.01, iterations=1000)
start_time_SGD = time.time()
model_SGD.fit(X_train, y_train)
end_time_SGD = time.time()
accuracy_SGD = model_SGD.evaluate_accuracy(X_test, y_test)
print()


print("Метод оптимизации: {model_RMSProp}")
model_RMSProp = RMSProp(learning_rate=0.01, decay_rate=0.9, epsilon=1e-7, iterations=1000)
start_time_RMSProp = time.time()
model_RMSProp.fit(X_train, y_train)
end_time_RMSProp = time.time()
accuracy_RMSProp = model_RMSProp.evaluate_accuracy(X_test, y_test)
print()

print("Метод оптимизации: {model_Nadam}")
model_Nadam = Nadam(learning_rate=0.01, beta1=0.9, beta2=0.999, epsilon=1e-8, iterations=1000)
start_time_Nadam = time.time()
model_Nadam.fit(X_train, y_train)
end_time_Nadam = time.time()
accuracy_Nadam = model_Nadam.evaluate_accuracy(X_test, y_test)
print()

# Создадим таблицу для сравнения методов
data = {'Метод': ['SGD', 'RMSProp', 'Nadam'],
        'Точность': [accuracy_SGD, accuracy_RMSProp, accuracy_Nadam],
        'Время работы': [end_time_SGD - start_time_SGD, end_time_RMSProp - start_time_RMSProp, end_time_Nadam - start_time_Nadam]}

# Выведем таблицу
comparison_table = pd.DataFrame(data)
print(comparison_table)


Метод оптимизации: {model_SGD}
Iteration 0: Cost 0.6931471805599454
Iteration 100: Cost 0.45908130152855275
Iteration 200: Cost 0.36677138337919923
Iteration 300: Cost 0.3150119513424012
Iteration 400: Cost 0.2802327747754729
Iteration 500: Cost 0.25446851085833455
Iteration 600: Cost 0.2342393427021266
Iteration 700: Cost 0.217745167515065
Iteration 800: Cost 0.20393893981888678
Iteration 900: Cost 0.19215774021617862

Метод оптимизации: {model_RMSProp}
Iteration 0: Cost 0.525
Iteration 100: Cost 0.04496702226353591
Iteration 200: Cost 0.04527152713307672
Iteration 300: Cost 0.0452734423830569
Iteration 400: Cost 0.04527344892722721
Iteration 500: Cost 0.045273448849999884
Iteration 600: Cost 0.04527344885016306
Iteration 700: Cost 0.04527344885016438
Iteration 800: Cost 0.04527344885016436
Iteration 900: Cost 0.04527344885016436

Метод оптимизации: {model_Nadam}
Iteration 0: Cost 0.525
Iteration 100: Cost 0.04514873399083481
Iteration 200: Cost 0.04490368989770834
Iteration 300: Cost

1. Точность моделей:
   - Метод оптимизации SGD показал наивысшую точность среди всех методов, с точностью 90% на тестовом наборе данных.
   - RMSProp и Nadam показали немного более низкую точность, соответственно 85% и 80%.

2. Время работы:
   - SGD демонстрировал наименьшее время обучения модели, что может быть связано с простотой самого метода. Время работы составило приблизительно 0.1 секунды.
   - RMSProp и Nadam потребовали больше времени для сходимости, обучившись за приблизительно 0.12 и 0.13 секунды соответственно.

3. Сравнение методов:
   - SGD эффективно справился с задачей классификации и показал наилучший результат по точности предсказаний.
   - RMSProp и Nadam имеют немного более низкую точность, что может быть связано с особенностями сходимости и настройками гиперпараметров.